In [ ]:
!pip install scanpy squidpy --quiet
!pip install scikit-misc

ERROR: Could not find a version that satisfies the requirement SpatialDE2 (from versions: none)
ERROR: No matching distribution found for SpatialDE2


In [ ]:
# Import of libraries for working with spatial transcriptomics data
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import squidpy as sq
from matplotlib.colors import ListedColormap
import argparse

/usr/lib/python3.12/importlib/__init__.py:90: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  return _bootstrap._gcd_import(name[level:], package, level)
/usr/local/lib/python3.12/dist-packages/squidpy/gr/_utils.py:23: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  CAN_USE_SPARSE_ARRAY = Version(anndata.__version__) >= Version("0.11.0rc1")
/usr/local/lib/python3.12/dist-packages/anndata/__init__.py:70: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)


##Loading data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/Shareddrives/Spatial_Transcriptomics/data_after_qc/SN048_A121573_Rep1/adata_QC_filtered.normalized.h5ad"

print("Loading dataset...")

# The read_visium function automatically detects the folder containing 'filtered_feature_bc_matrix' and 'spatial'
# Load Visium (10x Genomics) data into an AnnData object
adata_qc = sc.read_h5ad(data_path)
print("Dataset loaded.")

Loading dataset...
Dataset loaded.


/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1798: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


# SpatialDE
Prepare the data.
SpatialDE expects a gene × cell matrix and the spatial coordinates of each cell/spot.

In [ ]:
conda create -n spatialde python=3.10
conda activate spatialde
!pip install SpatialDE

SyntaxError: invalid syntax (ipython-input-829706423.py, line 1)

In [ ]:
# Extract expression matrix (log-normalized)
expr = pd.DataFrame(    #??? adata_HVG without mitichondrial
    adata_HVG.X.toarray() if hasattr(adata.X, "toarray") else adata_HVG.X,
    index=adata_HVG.obs_names,
    columns=adata_HVG.var_names
)

# Extract spatial coordinates (usually stored in adata.obsm["spatial"])
coords = pd.DataFrame(
    adata_HVG.obsm["spatial"],
    index=adata_HVG.obs_names,
    columns=["x", "y"]
)

data_for_spatialde = pd.concat([coords, expr], axis=1)